In [11]:
# preprocessing.py
import cv2
import numpy as np
import os
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

DIR = 'MalariaDrugImagesGHS'
DIM = 128

def resize_images(dir_, size):
    count = len(os.listdir(dir_))
    images = np.zeros([count, size, size], dtype='uint8')

    # Read and resize images
    scaler = StandardScaler()
    for (i,image) in zip(range(len(os.listdir(dir_))), os.listdir(dir_)):
        im_arr = cv2.imread(os.path.join(dir_, image))
        im_arr = cv2.cvtColor(im_arr, cv2.COLOR_BGR2GRAY)
        im_arr = cv2.resize(im_arr, (size, size))
        im_arr = scaler.fit_transform(im_arr)
        images[i] = im_arr
    images = images.reshape(count, -1)
    return images

def pca(images, components):
    pca = PCA(n_components= components)
    pca.fit(images)
    lower_dimensional_data = pca.fit_transform(images)
    expected_variance = pca.explained_variance_ratio_.cumsum()[-1]
    return (lower_dimensional_data, expected_variance)

In [12]:
images = resize_images(DIR, 128)
lower_dimensional_data, variance = pca(images, 2)

In [9]:
#save features to CSV
from numpy import savetxt
savetxt('data.csv', lower_dimensional_data, delimiter=',')

In [1]:
#!/usr/bin/env python
# coding: utf-8


import cv2
import numpy as np
import os
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

DIR = 'MalariaDrugImagesGHS'
DIM = 128

count = len(os.listdir(DIR))
images = np.zeros([count, DIM, DIM], dtype='uint8')

# Read and resize images
scaler = StandardScaler()
for (i,image) in zip(range(len(os.listdir(DIR))), os.listdir(DIR)):
    im_arr = cv2.imread(os.path.join(DIR, image))
    im_arr = cv2.cvtColor(im_arr, cv2.COLOR_BGR2GRAY)
    im_arr = cv2.resize(im_arr, (DIM, DIM))
    im_arr = scaler.fit_transform(im_arr)
    plt.imshow(im_arr)
    images[i] = im_arr

images = images.reshape(count, -1)

'''
pca = PCA(n_components=200)
pca.fit(images)

'''
# pca = PCA(.99)
# lower_dimensional_data = pca.fit_transform(images)
# pca.n_components_


# In[89]:

lower_dimensional_data = None
for var in [0.9]: #, 0.9, 0.85, 0.8, 0.75, 0.3]:
    pca = PCA(var)
    pca.fit(images)
    lower_dimensional_data = pca.fit_transform(images)
    approximation = pca.inverse_transform(lower_dimensional_data)
    plt.figure(figsize=(8,4));

    # Original Image
    plt.subplot(1, 2, 1);
    plt.imshow(images[1].reshape(DIM,DIM),
                  cmap = plt.cm.gray, interpolation='nearest',
                  clim=(0, 255));
    plt.xlabel(str(images[0].shape[0]) + ' components', fontsize = 14)
    plt.title('Original Image', fontsize = 20);

    # 154 principal components
    plt.subplot(1, 2, 2);
    plt.imshow(approximation[1].reshape(DIM, DIM),
                  cmap = plt.cm.gray, interpolation='nearest',
                  clim=(0, 255));
    plt.xlabel(str(pca.n_components_) + ' components', fontsize = 14)
    plt.title(str(var*100) + '% of Explained Variance', fontsize = 20);
    plt.show()
    print('Keeping ' +str(var*100) +' percent of the original data requires: ' + str(pca.n_components_) + ' bytes')



<Figure size 640x480 with 1 Axes>

<Figure size 800x400 with 2 Axes>

Keeping 90.0 percent of the original data requires: 25 bytes


In [63]:
lower_dimensional_data.shape

(30, 25)

In [65]:
#save features to CSV
from numpy import savetxt
savetxt('data.csv', lower_dimensional_data, delimiter=',')

In [46]:
from qiskit import QuantumCircuit, execute, Aer, IBMQ
IBMQ.

In [53]:

# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
import math
# Loading your IBM Q account(s)

# import settings

import os


class QBitsLoader():
    
    def __init__(self, online=False):
        self.online = online

        if self.online:
            TOKEN = os.getenv('TOKEN')
            IBMQ.save_account(TOKEN, overwrite=True)
            provider = IBMQ.load_account()
            self.backends = provider.backends()
            self.backends.append(Aer.backends())
        else:
            print('[INFO] Offline mode: only simulators can be accessed')
            self.backends = Aer.backends()

    def get_backends(self):
        """
            Returns a list of tuples with the available backends and their number of qubits
        """
        return [backend.name() for backend in self.backends]

    def get_backends_with_qubits(self):
        """
            Returns a list of tuples with the available backends and their number of qubits
        """
        return [(backend.name(), backend.configuration().n_qubits,) for backend in self.backends]
    
    def find_backend(self, name):
        """
            Finds a backend by its name
        """
        for backend in self.backends:
            if name == backend.name():
                return backend
        raise ValueError(f"Backend '{name}' not found")
    
    def get_backend_qubits(self, name):
        """
            Returns the number of qubits a certain quantum device can handle
        """
        return self.find_backend(name).configuration().n_qubits
    
    def pca_recommendation(self, dir_, name, size=128):
        qubits = self.get_backend_qubits(name)
        components = min(2**qubits, 512, len(dir_))
        if int(math.log2(components)) != components:
                components = 2**(int(math.log2(components)))
        image = resize_images(dir_, size)
        lower_dimensional_data, variance = pca(images, components)
        print('[INFO] Using ' + str(components) + ' number of components for PCA')
        print('[INFO] This preserves ' + str(variance) + '% of the original variance in the images')
        return lower_dimensional_data
    
    

if __name__ == "__main__":
    q_bits_loader = QBitsLoader(False)
    
    print(q_bits_loader.get_backends())
    print(q_bits_loader.get_backends_with_qubits())
    b = q_bits_loader.find_backend('unitary_simulator')

[INFO] Offline mode: only simulators can be accessed
['qasm_simulator', 'statevector_simulator', 'unitary_simulator']
[('qasm_simulator', 29), ('statevector_simulator', 29), ('unitary_simulator', 14)]


In [57]:
import numpy as np
import math
import matplotlib.pyplot as plt 
from qiskit.visualization import plot_histogram, plot_bloch_multivector

from qiskit.aqua.components.initial_states import Custom
from qiskit import ClassicalRegister, Aer, execute, QuantumCircuit, QuantumRegister

class QuantumData:
    def __init__(self, dir_, device=None):
        self.dir_ = dir_
        self.device = device
    def choose_device(self, name):
        self.device = name
    def transform(self):
        qbit_loader = QBitsLoader(False)
        lower_dimensional_data = qbit_loader.pca_recommendation(self.dir_, self.device)
        quantum_data = []
        for image,feature in zip(os.listdir(DIR),lower_dimensional_data):
            vector = np.array(feature)
            length = vector.shape[0]
            if int(math.log2(length)) != length:
                pad_length = 2**(int(math.log2(length))+1)
                vector = np.pad(vector, (0, pad_length - length), mode='constant')
            length = vector.shape[0]

            cr, qbits = self.circuit(vector)
        #     print(list(range(qbits)))
        #     cr.add_register(ClassicalRegister(qbits))
        #     cr.measure(list(range(qbits)), list(range(qbits)))

        #     simulator = Aer.get_backend('qasm_simulator')
        #     counts = execute(cr, backend=simulator, shots=1024).result().get_counts()
        #     plot_histogram(counts)
        #     simulator = Aer.get_backend('statevector_simulator')
        #     result = execute(cr, backend = simulator).result()
        #     statevector = result.get_statevector()
        #     plot_bloch_multivector(statevector)
        #     cr.draw(output = 'mpl')
            quantum_data.append(cr)

        #     plt.show()

            # 
            # print(dir(cr))
        return quantum_data
        
    def circuit(self, vec):
        qbits =  int(math.log2(length))
        cct = Custom(qbits, state_vector=vec)
        return cct.construct_circuit('circuit'), qbits

In [58]:
data = QuantumData(DIR, 'qasm_simulator')
d = data.transform()

[INFO] Offline mode: only simulators can be accessed
[INFO] Using 16 number of components for PCA
[INFO] This preserves 0.6574014609210423% of the original variance in the images


In [59]:
len(d)

30

In [61]:
print(d[0])

                                                                 »
q_0: |0>─────────────────────────────────────────────────────────»
                                                                 »
q_1: |0>─────────────────────────────────────────────────────────»
                                                                 »
q_2: |0>─────────────────────────────────────────────────────────»
                                    ┌───┐┌─────────────────┐┌───┐»
q_3: |0>────────────────────────────┤ X ├┤ U3(0.72867,0,0) ├┤ X ├»
        ┌───────────┐┌─────────────┐└─┬─┘└─────────────────┘└─┬─┘»
q_4: |0>┤ U3(0,0,0) ├┤ U1(-1.7671) ├──■───────────────────────■──»
        └───────────┘└─────────────┘                             »
«                                                                   »
«q_0: ──────────────────────────────────────────────────────────────»
«                                                                   »
«q_1: ───────────────────────────────────────────────

In [6]:
print(quantum_data[0])

                                                                       »
q_0: |0>───────────────────────────────────────────────────────────────»
                                                                       »
q_1: |0>───────────────────────────────────────────────────────────────»
                                                                       »
q_2: |0>───────────────────────────────────────────────────────────────»
                                          ┌───┐┌─────────────────┐┌───┐»
q_3: |0>──────────────────────────────────┤ X ├┤ U3(0.58275,0,0) ├┤ X ├»
        ┌────────────────┐┌──────────────┐└─┬─┘└─────────────────┘└─┬─┘»
q_4: |0>┤ U3(1.1158,0,0) ├┤ U1(-0.98175) ├──■───────────────────────■──»
        └────────────────┘└──────────────┘                             »
«                                                                      »
«q_0: ─────────────────────────────────────────────────────────────────»
«                                                  

In [3]:
counts.values()

dict_values([1, 59, 404, 68, 35, 5, 64, 105, 95, 2, 186])

In [4]:
%matplotlib

Using matplotlib backend: Qt5Agg
